In [2]:
pip install requests pandas googlemaps osmnx folium geopy


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 5.0 MB/s eta 0:00:00
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40715 sha256=d07be7284b019ccbfda4cdf760eff00638ae779a0acef15da7d5ead3becd97a1
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [3]:
import googlemaps
import pandas as pd
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
import osmnx as ox
import networkx as nx
import folium
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
def fetch_route_osmnx(origin_coords, destination_coords, travel_mode="walk"):
    """
    Fetches a route between origin and destination coordinates using OSMnx.

    Args:
        origin_coords (tuple): (latitude, longitude) of the origin.
        destination_coords (tuple): (latitude, longitude) of the destination.
        travel_mode (str): Mode of travel. Options: "walk", "bike", "drive".

    Returns:
        list: List of (latitude, longitude) tuples representing the route.
    """
    # Get the graph for the specified travel mode
    mode_to_network_type = {
        "walk": "walk",
        "bike": "bike",
        "drive": "drive",
    }
    network_type = mode_to_network_type.get(travel_mode, "walk")

    # Load the graph around the origin
    G = ox.graph_from_point(origin_coords, dist=2000, network_type=network_type)

    # Find the nearest nodes to the origin and destination
    origin_node = ox.distance.nearest_nodes(G, origin_coords[1], origin_coords[0])
    destination_node = ox.distance.nearest_nodes(G, destination_coords[1], destination_coords[0])

    # Calculate the shortest path
    route = nx.shortest_path(G, origin_node, destination_node, weight="length")

    # Get the coordinates of the route
    route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
    return route_coords

In [5]:
def process_route_data(route_coords):
    """
    Converts route coordinates into a Pandas DataFrame.

    Args:
        route_coords (list): List of (latitude, longitude) tuples.

    Returns:
        pd.DataFrame: Tabular representation of route steps.
    """
    # Create DataFrame from route coordinates
    route_df = pd.DataFrame(route_coords, columns=["latitude", "longitude"])
    return route_df

# Define origin and destination coordinates
origin = (-7.2946612, 112.7380407)  # Example: Empire State Building
destination = (-7.2628945, 112.7411374)  # Example: Astoria Park

# Fetch and process the route
route_coords = fetch_route_osmnx(origin, destination, travel_mode="walk")
route_df = process_route_data(route_coords)

print(route_df.head())  # Display the first few rows of the route

# Save to CSV for future use
route_df.to_csv("route_data_osmnx.csv", index=False)

   latitude   longitude
0 -7.294157  112.738049
1 -7.294074  112.738224
2 -7.292868  112.737626
3 -7.292457  112.737422
4 -7.292400  112.737415


In [34]:
def plot_route_on_map(route_coords):
    """
    Plot the route on a map using Folium.

    Args:
        route_coords (list): List of (latitude, longitude) tuples.
    """
    # Create a map centered at the midpoint of the route
    mid_idx = len(route_coords) // 2
    route_map = folium.Map(location=route_coords[mid_idx], zoom_start=14)

    # Add the route to the map
    folium.PolyLine(route_coords, color="blue", weight=5).add_to(route_map)

    # Mark start and end points
    folium.Marker(route_coords[0], popup="Start", icon=folium.Icon(color="green")).add_to(route_map)
    folium.Marker(route_coords[-1], popup="End", icon=folium.Icon(color="red")).add_to(route_map)

    return route_map

route_map = plot_route_on_map(route_coords)
route_map.save("route_map_osmnx.html")
print("Route map saved as route_map_osmnx.html.")

Route map saved as route_map_osmnx.html.


In [35]:
def generate_route_instructions(route_coords, G):
    """
    Generates basic textual instructions for a route based on OSMnx graph.

    Args:
        route_coords (list): List of (latitude, longitude) tuples.
        G (networkx.MultiDiGraph): OSMnx graph.

    Returns:
        list: List of textual instructions for the route.
    """
    instructions = []
    for i in range(len(route_coords) - 1):
        # Get the current and next nodes
        current_node = ox.distance.nearest_nodes(G, route_coords[i][1], route_coords[i][0])
        next_node = ox.distance.nearest_nodes(G, route_coords[i + 1][1], route_coords[i + 1][0])

        # Get edge data between the current and next node
        edge_data = G.get_edge_data(current_node, next_node, default={})

        # Extract street name or set to "Unknown"
        street_name = edge_data.get(0, {}).get("name", "Unknown")

        # Approximate an instruction
        instruction = f"Go towards {street_name}" if street_name else "Continue straight"
        instructions.append(instruction)
    return instructions

# Define origin and destination coordinates
origin = (-7.2946612, 112.7380407)  # Example: Empire State Building
destination = (-7.2628945, 112.7411374)  # Example: Astoria Park

# Fetch route using OSMnx
travel_mode = "walk"
G = ox.graph_from_point(origin, dist=2000, network_type=travel_mode)
origin_node = ox.distance.nearest_nodes(G, origin[1], origin[0])
destination_node = ox.distance.nearest_nodes(G, destination[1], destination[0])
route = nx.shortest_path(G, origin_node, destination_node, weight="length")
route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]

# Generate instructions
instructions = generate_route_instructions(route_coords, G)

# Create a DataFrame
route_df = pd.DataFrame({
    "start_location": route_coords[:-1],
    "end_location": route_coords[1:],
    "instruction": instructions,
    "distance": [nx.shortest_path_length(G, route[i], route[i+1], weight="length") for i in range(len(route) - 1)],
    "duration": [0.02 * nx.shortest_path_length(G, route[i], route[i+1], weight="length") for i in range(len(route) - 1)],  # Assuming average walking speed
})

# Encode text instructions as numerical data
label_encoder = LabelEncoder()
route_df['instruction_encoded'] = label_encoder.fit_transform(route_df['instruction'])

# Features: Encoded instructions, distance, and duration
X = route_df[['instruction_encoded', 'distance', 'duration']].values

# Labels: Predict next end location (latitude, longitude)
y = route_df['end_location'].apply(lambda loc: (loc[0], loc[1])).tolist()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training features (X_train):", X_train.shape)
print("Training labels (y_train):", len(y_train))

Training features (X_train): (28, 3)
Training labels (y_train): 28


In [81]:
# Define the model to predict distances for ranking
def create_model():
    model = tf.keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=(3,)),  # Input: [latitude, longitude, placeholder]
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='linear')  # Output: Predicted distance
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Create the model
model = create_model()
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                     │ (None, 64)                  │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,369 (9.25 KB)

 Trainable params: 2,369 (9.25 KB)

 Non-trainable params: 0 (0.00 B)

In [82]:
# Example: Generate dummy training data
np.random.seed(42)
n_samples = 100
latitudes = np.random.uniform(-90, 90, n_samples)
longitudes = np.random.uniform(-180, 180, n_samples)
placeholders = np.zeros(n_samples)  # Placeholder feature

# Combine features
X_train = np.column_stack((latitudes, longitudes, placeholders))

# Generate labels: distances from a fixed origin (e.g., [0, 0])
origin = (0.0, 0.0)
y_train = np.array([
    geodesic((lat, lon), origin).meters for lat, lon in zip(latitudes, longitudes)
])

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=8,
    verbose=1
)

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test MAE: {mae}")

Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - loss: 107925037121536.0000 - mae: 9881303.0000 - val_loss: 126314761682944.0000 - val_mae: 10688787.0000
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 115006783881216.0000 - mae: 10202696.0000 - val_loss: 126314392584192.0000 - val_mae: 10688772.0000
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 122539116331008.0000 - mae: 10429365.0000 - val_loss: 126313964765184.0000 - val_mae: 10688754.0000
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 116361325969408.0000 - mae: 10105234.0000 - val_loss: 126313520168960.0000 - val_mae: 10688736.0000
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 114355584630784.0000 - mae: 10170118.0000 - val_loss: 126313025241088.0000 - val_mae: 10688716.0000
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 121844740915200.0000 - mae: 10519398.0000 - val_loss: 126312438038528.0000 - val_mae: 10688692.0000
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1023

In [83]:
from geopy.distance import geodesic

# Function to optimize route from start to farthest point
def optimize_route(addresses, coordinates):
    """
    Addresses: List of dicts with latitude and longitude
    Start_point: Dict with latitude and longitude
    """

    try:
        print("Starting optimization for addresses:", addresses)

        # Gunakan alamat pertama sebagai titik awal
        start_point = {"address": addresses[0], "latitude": coordinates[0][0], "longitude": coordinates[0][1]}
        print("Start Point:", start_point)

        # Calculate distances from start_point using geodesic
        distances = [
            {
                "index": idx,
                "address": addresses[idx],
                "distance": geodesic(
                    (start_point["latitude"], start_point["longitude"]),
                    coord
                ).meters,
            }
            for idx, coord in enumerate(coordinates[1:], start=1)
        ]

        # Urutkan berdasarkan jarak
        sorted_distances = sorted(distances, key=lambda x: x["distance"])
        print("Sorted Distances:", sorted_distances)

        # Tensor input untuk prediksi model
        input_tensor = np.array([[lat, lon, 0] for lat, lon in coordinates], dtype=np.float32)
        print("Input Tensor Shape:", input_tensor.shape)

        # Muat model TensorFlow
        predictions = model.predict(input_tensor)

        # Gabungkan prediksi dengan hasil jarak
        for idx, prediction in enumerate(predictions):
            if idx > 0:  # Skip start_point
                sorted_distances[idx - 1]["prediction"] = prediction[0]

        # Urutkan berdasarkan jarak dan prediksi (jika diperlukan)
        final_sorted = sorted(sorted_distances, key=lambda x: x["distance"])
        print("Final Sorted:", final_sorted)

        # Susun rute akhir
        optimized_route = [start_point["address"]] + [item["address"] for item in final_sorted]
        print("Optimized Route:", optimized_route)

        return {
            "optimizedRoute": optimized_route
        }

    except Exception as e:
        print("Error in optimize_route:", str(e))
        raise e

In [91]:
# Example usage
addresses = [
    "UPN Veteran Jawa Timur",
    "Muhammad Cheng Hoo Mosque",
    "UNAIR Kampus C",
    "ITS"

]

coordinates = [
    (-7.333721199999999, 112.7883247),
    (-7.251958099999999, 112.7468025),
    (-7.268716200000001, 112.7842396),
    (-7.2801485, 112.7977885)
]

# start_point = {"latitude": -7.3336995, "longitude": 112.7698706}

resut = optimize_route(addresses, coordinates)

# print("\nOptimized Route Results:")
# print("Start Point:", result["startPoint"])
# print("Nearest Point:", result["nearestPoint"])
# print("Farthest Point:", result["farthestPoint"])
# print("Optimized Route:", result["optimizedRoute"])

Starting optimization for addresses: ['UPN Veteran Jawa Timur', 'Muhammad Cheng Hoo Mosque', 'UNAIR Kampus C', 'ITS']
Start Point: {'address': 'UPN Veteran Jawa Timur', 'latitude': -7.333721199999999, 'longitude': 112.7883247}
Sorted Distances: [{'index': 3, 'address': 'ITS', 'distance': 6016.178444278844}, {'index': 2, 'address': 'UNAIR Kampus C', 'distance': 7203.184921933534}, {'index': 1, 'address': 'Muhammad Cheng Hoo Mosque', 'distance': 10138.40480647809}]
Input Tensor Shape: (4, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Final Sorted: [{'index': 3, 'address': 'ITS', 'distance': 6016.178444278844, 'prediction': 29158.46}, {'index': 2, 'address': 'UNAIR Kampus C', 'distance': 7203.184921933534, 'prediction': 29170.473}, {'index': 1, 'address': 'Muhammad Cheng Hoo Mosque', 'distance': 10138.40480647809, 'prediction': 29175.719}]
Optimized Route: ['UPN Veteran Jawa Timur', 'ITS', 'UNAIR Kampus C', 'Muhammad Cheng Hoo Mosque']


In [ ]:
model.save('/content/gdrive/My Drive/Colab Notebooks/Capstone/model.h5')

In [87]:
model.export('saved_model')

Saved artifact at 'saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 3), dtype=tf.float32, name='keras_tensor_15')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  136300885362112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136300849749456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136300849757728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136300789169696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136300789165648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136300888253968: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [30]:
pip install tensorflowjs

In [88]:
#!tensorflowjs_converter --input_format keras new_model.h5 ./output_model
!tensorflowjs_converter --input_format=tf_saved_model saved_model neural_network_model_tfjs --skip_op_check
!zip -r model_json_baru.zip neural_network_model_tfjs

2024-12-11 08:17:54.851185: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733905074.892395   70053 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733905074.905657   70053 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
2024-12-11 08:17:59.998299: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
I0000 00:00:1733905080.084884   70053 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00